##Data Tranformation

####Create Table

In [0]:
---DROP TABLE IF EXISTS digital_funnel.silver.FunnelData;
---DROP TABLE IF EXISTS digital_funnel.silver.CatalogData;

In [0]:
CREATE TABLE IF NOT EXISTS digital_funnel.silver.funneldata
(
  cliente_id STRING,
  app STRING,
  stage STRING,
  stage_date DATE,
  proceso_completo INT,
  ingestion_time TIMESTAMP
)
LOCATION 'abfss://silver@firstprojectdlucext.dfs.core.windows.net/funneldata/';

In [0]:
CREATE TABLE IF NOT EXISTS digital_funnel.silver.catalogdata
(
  cliente_id STRING,
  nombre STRING,
  edad INT,
  estado STRING,
  correo STRING,
  ingestion_time TIMESTAMP,
  updated_date TIMESTAMP 
)
LOCATION 'abfss://silver@firstprojectdlucext.dfs.core.windows.net/catalogdata/';

####Edit columns

In [0]:
CREATE OR REPLACE TEMP VIEW FunnelIncrement
AS
SELECT ID_cliente AS cliente_id,
       APP AS app,
       stage,
       Date AS stage_date,
       Proceso_completo AS proceso_completo
FROM digital_funnel.bronze.usuarios_banco_con_funnel;

In [0]:
CREATE OR REPLACE TEMP VIEW CatalogIncrement
AS
SELECT ID_cliente AS cliente_id,
       Nombre AS nombre,
       Correo AS correo,
       Edad AS edad,
       Estado AS estado
FROM digital_funnel.bronze.clientes;

####Ingestig into silver 

In [0]:
MERGE INTO digital_funnel.silver.funneldata  AS tgt
USING FunnelIncrement AS src
ON (tgt.cliente_id = src.cliente_id AND tgt.stage = src.stage)
WHEN NOT MATCHED THEN
  INSERT (tgt.cliente_id, tgt.app, tgt.stage, tgt.stage_date, tgt.proceso_completo, tgt.ingestion_time)
  VALUES (src.cliente_id, src.app, src.stage, src.stage_date, src.proceso_completo, current_timestamp)
---PARTITIONED BY (race_year)

In [0]:
MERGE INTO digital_funnel.silver.catalogdata  AS tgt
USING CatalogIncrement AS src
ON (tgt.cliente_id = src.cliente_id)
WHEN MATCHED THEN
  UPDATE SET tgt.nombre = src.nombre,
             tgt.correo = src.correo,
             tgt.edad = src.edad,
             tgt.estado = src.estado,
             tgt.updated_date = current_timestamp
WHEN NOT MATCHED THEN
  INSERT (tgt.cliente_id, tgt.nombre, tgt.correo, tgt.edad, tgt.estado, tgt.ingestion_time)
  VALUES (src.cliente_id, src.nombre, src.correo, src.edad, src.estado, current_timestamp)

In [0]:
SELECT *
FROM digital_funnel.silver.CatalogData;